# MODELOS DE ENTRENAMIENTO DE MACHINE LEARNING    

## Logistic Regression


In [ ]:
from src.utils import get_sample_data

df_sample = get_sample_data()

In [ ]:
from src.feature_engineerings import create_pattern_features

# Aplicar  
df = create_pattern_features(df_sample)

# Análisis de patrones
pattern_analysis = df.groupby(['off_hours', 'suspicious_frequency'])['isFraud'].mean()
print("Análisis de patrones sospechosos:")
print(pattern_analysis)

In [ ]:
from src.feature_engineerings import encode_categorical_features

# Aplicar
df_encode, label_encoder = encode_categorical_features(df)
print("Variables categóricas codificadas exitosamente")
print(df_encode[[col for col in df_encode.columns if 'hour' in col]].head())




In [ ]:
# Logistic Regression para detección de fraude
from src.feature_engineerings import  scale_features
from src.feature_engineerings import  select_important_features
from src.models import logistic_regression



# Aplicar y escalar dataset
X_train, X_test, y_train, y_test, scaler = scale_features(df_encode)

feature_importance, selected_features, selector = select_important_features(X_train, y_train)
print("Características seleccionadas:")
#print(selected_features)

# Crear dataset final optimizado
X_train_final = X_train[selected_features]
X_test_final  = X_test[selected_features]




lr_model = logistic_regression(X_train_final, y_train, X_test_final, y_test)

print("Modelo de regresión logística entrenado y evaluado.")
#print("Características seleccionadas:", lr_model)  # Asumiendo que el cuarto elemento es 'selected_features'
selected_features = X_train.columns[selector.get_support()]
print(selected_features.tolist())







NOTA: LA REGRESION LOGISTICA TIENE UN RECALL ALTISIMO PERO CON MUCHOS FALSOS POSITIVOS TOCARIA USAR TECNICAS DE BALANCEO DEL DATASET COMO SMOTE PARA LOGRAR UN RESULTADO CON MAS PRESICION

# Random Forest

Random Forest es un modelo de aprendizaje supervisado basado en ensembles de árboles de decisión. Combina múltiples árboles para reducir la varianza y mejorar la generalización. Usa bootstrap aggregation (bagging) y selecciona aleatoriamente subconjuntos de features en cada árbol.

🧪 Ideal para:
    - Clasificación con datos tabulares
    - Manejo de overfitting
    - Datasets desequilibrados con class_weight='balanced'






In [ ]:
# Random Forest para detección de fraude
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report
import pandas as pd

# Configurar modelo
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_features = 'sqrt',
    min_samples_leaf = 4,
    min_samples_split = 2,
    max_depth=12,
    class_weight='balanced',
    random_state=42
)

# Entrenar
print("Entrenando Random Forest...")
rf_model.fit(X_train_final, y_train)

# Evaluar
y_pred = rf_model.predict(X_test_final)
y_prob = rf_model.predict_proba(X_test_final)[:, 1]
auc_score = roc_auc_score(y_test, y_prob)

print("Resultados:")
print("AUC Score:", auc_score)
print(classification_report(y_test, y_pred))

# Feature importance
importance_df = pd.DataFrame({
    'feature': X_train_final.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Top features importantes:")
print(importance_df.head(10))

# XGBoost 

(eXtreme Gradient Boosting) es una implementación optimizada de gradient boosting. Aprende secuencialmente minimizando una función de pérdida con regularización L1/L2 para prevenir overfitting. Usa árboles de decisión como base learners.



In [ ]:
# XGBoost para máximo rendimiento
import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix


# Calcular peso para balancear clases
normal_count = len(y_train[y_train==0])
fraud_count = len(y_train[y_train==1])
pos_weight = normal_count / fraud_count

# Configurar modelo
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    subsample = 0.8,
    reg_alpha = 0.5,
    reg_lambda = 2.0,
    gamma = 0,
    colsample_bytree = 1.0,
    max_depth=10,
    learning_rate=0.1,
    scale_pos_weight=pos_weight,
    random_state=42
)

# Entrenar
print("Entrenando XGBoost...")
xgb_model.fit(X_train_final, y_train)

# Evaluar
y_pred = xgb_model.predict(X_test_final)
# 💡 Nuevo threshold (por ejemplo, 0.4 en lugar de 0.5)
threshold = 0.3  # <-- Aquí puedes ir probando: 0.3, 0.4, 0.6, etc.
y_pred_custom = (y_prob > threshold).astype(int)

# Evaluación con el nuevo threshold
auc_score = roc_auc_score(y_test, y_prob)
print(f"Resultados XGBoost con threshold {threshold}:")
print("AUC Score:", auc_score)
print(confusion_matrix(y_test, y_pred_custom))
print(classification_report(y_test, y_pred_custom))
auc_score = roc_auc_score(y_test, y_prob)

print("Modelo XGBoost entrenado y evaluado.")
print("Características seleccionadas:")
print(xgb_model.feature_names_in_)


In [ ]:
"""import joblib

# Guardar modelo entrenado
joblib.dump(xgb_model, 'modelo_xgboost_fraude.pkl')

# Guardar scaler y features seleccionadas si los usaste
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(selected_features, 'features_seleccionadas.pkl')

# Guardar threshold
with open('threshold.txt', 'w') as f:
    f.write(str(0.3))"""


In [ ]:
import joblib

# === 1. Guardar modelo entrenado ===
joblib.dump(xgb_model, 'modelo_xgboost_fraude.pkl')

# === 2. Guardar scaler usado para normalización ===
joblib.dump(scaler, 'scaler.pkl')

# === 3. Guardar selector de features importantes ===
joblib.dump(selector, 'selector_kbest.pkl')

# === 4. Guardar nombres de las features seleccionadas (para verificar columnas en producción) ===
joblib.dump(selected_features, 'features_seleccionadas.pkl')

joblib.dump(X_train, 'X_train_columns.pkl')  # Guarda las columnas en orden correcto


# === 5. Guardar threshold personalizado (si tu modelo usa un umbral ajustado para clasificar fraude) ===
with open('threshold.txt', 'w') as f:
    f.write(str(0.3))  # Cambiá 0.3 por tu valor real si lo tenés ajustado

print("✅ Todo fue guardado exitosamente para producción.")




## DESPLIEGUE ANTIFRAUDE MODELO XGBOOST

In [ ]:
import pandas as pd
import joblib

# === Cargar componentes entrenados ===
model = joblib.load('modelo_xgboost_fraude.pkl')
scaler = joblib.load('scaler.pkl')
selector = joblib.load('selector_kbest.pkl')
X_train_col = joblib.load('X_train_columns.pkl')  # Necesitás guardar esto en tu entrenamiento

# === Crear un diccionario con la estructura completa ===
data_prueba_dict = {

    'step': 250,               # Paso del tiempo alto (transacción tardía en el día)
    'amount': 9800.0,          # Monto alto y cerca de un umbral típico de reporte
    'oldbalanceOrg': 9800.0,   # El cliente transfiere casi todo su saldo
    'newbalanceOrig': 0.0,     # Queda sin saldo
    'oldbalanceDest': 0.0,     # Cuenta destino vacía
    'newbalanceDest': 9800.0,  # Recibe todo el dinero
    'hour_of_day': 3,          # De madrugada
    'off_hours': 1,            # Horario fuera de lo normal
    'day_of_week': 0,          # Lunes
    'weekend_activity': 0,     # No es fin de semana
    'amount_frequency': 120,   # Monto repetido muchas veces
    'suspicious_frequency': 1, # Actividad sospechosa
    'structured_amount': 1,    # Monto “estructurado” cerca de límites
    'dest_diversity': 20,      # Transfiere a muchos destinos
    'high_dest_diversity': 1,  # Fan-out alto
    'type_encoded': 4,         # TRANSFER (alto riesgo)
    'type_CASH_IN': 0,
    'type_CASH_OUT': 0,
    'type_DEBIT': 0,
    'type_PAYMENT': 0,
    'type_TRANSFER': 1,        # Es una transferencia
    'type_fraud_rate': 0.5,    # Alta tasa histórica de fraude en este tipo
    'hour_sin': -0.99,
    'hour_cos': 0.14
}




# === Convertir a DataFrame y alinear columnas ===
data_prueba = pd.DataFrame([data_prueba_dict])


# Reordenar columnas exactamente como en el entrenamiento
data_prueba = data_prueba[X_train_col.columns]  # OJO: necesitas haber guardado X_train.columns
#print("Columnas que espera el scaler:", list(X_train.columns))
#print("Columnas que le estoy pasando:", list(data_prueba.columns))

data_scaled = scaler.transform(data_prueba)

# === Selección de features importantes ===
data_selected = selector.transform(data_scaled)

# === Predicción ===
prediccion = model.predict(data_selected)
probabilidad = model.predict_proba(data_selected)[0][1]

# === Mostrar resultado ===
print("¿Es fraude?:", "✅ SÍ" if prediccion[0] == 1 else "❌ NO")
print(f"Probabilidad de fraude: {probabilidad:.2%}")


